In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector
from datetime import datetime

# Define the base URL
base_url = "https://www.maparatunisie.tn/categorie-produit/visages/page/{}/?_paged={}"

# Function to scrape a page and return a list of products
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.find_all('div', class_='product-small')
    else:
        print("La requête a échoué. Code d'état :", response.status_code)
        return []

# Function to extract product information from a product element
def extract_product_info(product):
    # Check if 'img' tag exists
    img_tag = product.find('img')
    if img_tag:
        # Extract 'data-src' attribute if present, else set to None
        image_link = img_tag.get('data-src', None)
    else:
        image_link = None

    product_name = product.find('p', class_='name').text.strip()

    # Handle NaN values for prices
    bdi_tags = product.find_all('bdi')
    prices = [float(bdi_tag.get_text(strip=True).replace("TND", "").replace(",", ".")) for bdi_tag in bdi_tags if bdi_tag.get_text(strip=True)]
    min_price = min(prices) if prices else None
    
    # Replace NaN with None (a valid Python value)
    if pd.isna(min_price):
        min_price = None

    product_link = product.find('a', class_='woocommerce-LoopProduct-link')['href']
    date_prix = datetime.today().strftime('%Y-%m-%d')

    return {
        'product_name': product_name,
        'image_link': image_link,
        'product_price': min_price,
        'product_link': product_link,
        'date_prix': date_prix
    }

# Function to execute the scraping and insertion with debugging
def job():
    # Initialize page number
    page_number = 1

    # Create an empty DataFrame to store product data
    columns = ['product_name', 'image_link', 'product_price', 'product_link', 'date_prix']
    df_products = pd.DataFrame(columns=columns)

    # Main loop for scraping each page
    while True:
        # Construct the URL with the current page number
        url = base_url.format(page_number, page_number)
        print(f"Scraping de la page : {url}")

        # Scrape the page
        products = scrape_page(url)

        if not products:
            print("Aucun produit trouvé sur cette page. Fin du scraping.")
            break

        for product in products:
            # Extract product information
            row_data = extract_product_info(product)
            df_products = pd.concat([df_products, pd.DataFrame([row_data])], ignore_index=True)
            print(f"Produit extrait : {row_data}")

        # Increment the page number for the next iteration
        page_number += 1

    # Remove duplicate products based on the product link
    df_products.drop_duplicates(subset='product_link', inplace=True)
    print("Nettoyage des doublons terminé.")

    # MySQL database connection configuration
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'database': 'scrapping_db',
        'port': 3306
    }

    # Establish a connection to the MySQL server
    with mysql.connector.connect(**db_config) as connexion:
        # Create a cursor object to interact with the database
        with connexion.cursor() as curseur:
            try:
                # Iterate over each row in the DataFrame
                for _, row in df_products.iterrows():
                    # Check if the product already exists
                    curseur.execute("SELECT id_produit FROM produits WHERE lien = %s", (row['product_link'],))
                    existing_product_id = curseur.fetchone()

                    if existing_product_id:
                        # Product already exists, get the ID
                        product_id = existing_product_id[0]
                        print(f"Le produit existe déjà. ID: {product_id}")
                    else:
                        # Product does not exist, insert into the "produits" table
                        curseur.execute("""
                            INSERT INTO produits (nom, nom_site, image, lien)
                            VALUES (%s, %s, %s, %s)
                        """, (row['product_name'], 'maparatunisie.tn', row['image_link'], row['product_link']))

                        # Get the ID of the last inserted product
                        curseur.execute("SELECT LAST_INSERT_ID()")
                        product_id = curseur.fetchone()[0]
                        print(f"Nouveau produit inséré. ID: {product_id}")

                    # Insert into the "prix" table with the current date
                    curseur.execute("""
                        INSERT INTO prix (id_produit, prix, date_prix)
                        VALUES (%s, %s, %s)
                    """, (product_id, row['product_price'] if not pd.isna(row['product_price']) else None, row['date_prix']))
                    print(f"Prix ajouté pour le produit. ID: {product_id}")

                # Commit the changes to the database
                connexion.commit()
                print("Données insérées dans la base de données MySQL.")
            except Exception as e:
                print(f"Une erreur s'est produite : {e}")

    print("Fin du scraping et de l'insertion.")

# Manual Execution
if __name__ == "__main__":
    job()




    # Scheduled Execution
# Uncomment the following lines if you want to schedule the script
# schedule.every().day.at("00:00").do(job)
# while True:
#     schedule.run_pending()
#     time.sleep(1)


Scraping de la page : https://www.maparatunisie.tn/categorie-produit/visages/page/1/?_paged=1


Produit extrait : {'product_name': 'CETAPHIL Lotion Nettoyante Peaux Seches et Sensibles 500ml', 'image_link': None, 'product_price': 41.25, 'product_link': 'https://www.maparatunisie.tn/produit/cetaphil-lotion-nettoyante-500-ml/', 'date_prix': '2023-12-10'}
Produit extrait : {'product_name': 'CETAPHIL Lotion Nettoyante Peaux Seches et Sensibles 500ml', 'image_link': None, 'product_price': 41.25, 'product_link': 'https://www.maparatunisie.tn/produit/cetaphil-lotion-nettoyante-500-ml/', 'date_prix': '2023-12-10'}
Produit extrait : {'product_name': 'SVR Blur SPF50+ 50ml', 'image_link': 'https://www.maparatunisie.tn/wp-content/uploads/2020/12/svr-blur-400x400.jpg', 'product_price': 48.0, 'product_link': 'https://www.maparatunisie.tn/produit/svr-blur/', 'date_prix': '2023-12-10'}
Produit extrait : {'product_name': 'SVR Blur SPF50+ 50ml', 'image_link': 'https://www.maparatunisie.tn/wp-content/uploads/2020/12/svr-blur-400x400.jpg', 'product_price': 48.0, 'product_link': 'https://www.maparatu